Прогнозирование инсульта у пациентов на основе входных параметров

In [1]:
# импортируем необходимые библиотеки
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split 

Описание данных: 

*gender* - пол

*age*  - возраст

*hypertension* - страдает ли пациент гипертонией

*heart_disease* - есть ли болезни сердца

*ever_married* - был ли женат/замужем

*work_type*  - тип работы

*Residence_type* - проживает в городе или селе

*avg_glucose_level* - средний уровень глюкозы

*bmi* - индекс массы тела

*smoking_status* - информация о курении

*stroke* - целевая переменная инсульт был или нет 

In [2]:
# загрузим датасет
df = pd.read_csv('23.3 stroke_data.csv')
print(df.shape)
df.head()

(4000, 11)


gender  age  hypertension  heart_disease  ever_married      work_type  \
0    Male   73             0              0             1        Private   
1  Female   49             0              0             1        Private   
2    Male   58             0              0             1        Private   
3  Female   69             0              0             1  Self-employed   
4    Male   60             0              0             1        Private   

   Residence_type  avg_glucose_level        bmi   smoking_status  stroke  
0               1         143.509078  29.160236  formerly smoked       1  
1               0          85.230000  25.400000          Unknown       0  
2               0         197.174377  34.870606  formerly smoked       1  
3               1          99.680000  17.600000  formerly smoked       0  
4               0          69.200000  30.900000     never smoked       0

In [3]:
# посчитаем количества уникальных значений в df.stroke
df.stroke.value_counts()

0    2027
1    1973
Name: stroke, dtype: int64

0    2027
1    1973
Значения почти одинаковы, значит датасет сбалансирован

In [7]:
# проверим, ести ли пропущенные значения в датасете
# команда any проверяет все записи по каждой колонке, если найдет хоть один пропуск - выдаст True
df.isnull().any()

gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
smoking_status       False
stroke               False
dtype: bool

Пропущенных значений в столбцах нет

Изучим особенности данных прежде чем изучать модель: 
- может какие-то из переменных заполнены одинаковыми значениями
- дублируют друг друга
тогда их нужно предварительно удалить из датасета
или воспользоваться скелерами для стандартизации данных

In [4]:
# нарисуем график зависимости возраста от целевой переменной stroke
plt.plot(df['age'], df.stroke, 'ro') 
plt.xlabel('age')
plt.ylabel('stroke')
plt.title('Зависимость целевой переменной от возраста')
plt.show()

In [9]:
# подготовим данные для обучения
x = df.drop(['stroke'], axis=1)
y = df.stroke


In [13]:
# разделим данные на тренировочный и тестовый сеты в соотношении 70 на 30
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)


In [14]:
# вызовим модуль логистической регрессии
logreg = LogisticRegression()

In [15]:
# обучим модель на примере одной переменной "возраст"
train_cols = ['age']
logreg.fit(train_x[train_cols], train_y)

LogisticRegression()

In [16]:
# найдем предикт на тестовых данных
pred = logreg.predict(test_x[train_cols])
accuracy_score(test_y, pred)

0.7683333333333333

In [18]:
# выведем матрицу ошибок
confusion_matrix(test_y, pred)

array([[448, 156],
       [122, 474]], dtype=int64)

In [19]:
# добавим для обучения модели еще одну переменную 'heart_disease'
train_cols = ['age', 'heart_disease']
logreg.fit(train_x[train_cols], train_y)

LogisticRegression()

In [20]:
# найдем предикт на тестовых данных и выведем матрицу ошибок
pred = logreg.predict(test_x[train_cols])
print(accuracy_score(test_y, pred))
confusion_matrix(test_y, pred)

0.785


array([[457, 147],
       [111, 485]], dtype=int64)

In [ ]:
# predict_proba() – значения вероятностей для каждого наблюдения
# то есть выдать «степень уверенности» в ответе (вероятность)
# [0] - for class '0' 0.85219662 for class '0'
# [1] - for class '1' 0.14780338 for class '1'
logreg.predict_proba(test_x[train_cols])

array([[0.85219662, 0.14780338],
       [0.97878403, 0.02121597],
       [0.83988949, 0.16011051],
       ...,
       [0.94222814, 0.05777186],
       [0.99426236, 0.00573764],
       [0.9923954 , 0.0076046 ]])

Можно самостоятельно регулировать предсказания классов используя вероятность

In [21]:
# посмотрим, какие коэффициенты подобрала логистическая регрессия в решающей функции
logreg.coef_

array([[ 0.0945283 , -1.22020129]])

In [22]:
# выведем коэффициенты для признаков в модели
# + значит положительная корелляция к целевой переменной
# - значит отрицательная корреляция к целевой переменной
for col, i in zip(train_cols, logreg.coef_[0]):
    print(f'Коэф при колонке {col}={i}')

Коэф при колонке age=0.09452830228833398
Коэф при колонке heart_disease=-1.2202012869196788


In [23]:
# выведем значение свободного члена
logreg.intercept_

array([-5.43853816])

In [ ]:
# выведем качество модели логистической регрессии при обучении на тренировочной выборке
logreg.score(train_x[train_cols], train_y)

0.7725

In [24]:
# выведем параметры, которые использовались для обучения модели
logreg.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

Значения параметров можно менять для улучшения предсказательной способности модели.
В логистической регрессии чаше всего можно получить улучшения настроив параметры регуляризации
то есть штрафует модель за слишком высокие коэффициенты.
Это позволяет сделать модель более общей, не затачивать ее под тренировочную выборку.
За регуляризацию отвечает параметр
'penalty' - штрафует модель за слишком высокие коэффициенты
'C' - мощнесть регуляризации

In [68]:
# настроим уровень регуляризации и проведем обучение с новыми параметрами
logreg_tuned = LogisticRegression(penalty='l1', solver='saga')
logreg_tuned.fit(train_x[train_cols], train_y)

C:\Users\HP Z2\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(penalty='l1', solver='saga')

In [69]:
# найдем предикт на тестовых данных и выведем матрицу ошибок
pred2 = logreg_tuned.predict(test_x[train_cols])
print(accuracy_score(test_y, pred2))
confusion_matrix(test_y, pred2)

0.7858333333333334


array([[429, 175],
       [ 82, 514]], dtype=int64)

In [70]:
# выведем коэффициенты для признаков в модели
# + значит положительная корелляция к целевой переменной
# - значит отрицательная корреляция к целевой переменной
for col, i in zip(train_cols, logreg_tuned.coef_[0]):
    print(f'Коэф при колонке {col}={i}')

Коэф при колонке age=0.06469579142788978
Коэф при колонке heart_disease=-0.5193891259331939
